### Imports

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
import scipy.signal as sg
import matplotlib.pyplot as plt
from neuropy import plotting
import subjects

### Theta-OFF/ripple-ON (Toro) cells
- Szabo et al. 2022


In [ ]:
from neuropy.core import Neurons
from neuropy.utils.signal_process import filter_sig, ThetaParams
from neuropy.utils.mathutil import thresh_epochs

sessions = subjects.pf_sess()

for s, sess in enumerate(sessions[1:2]):
    neurons = sess.neurons_stable.get_neuron_type(["pyr", "mua"])
    fs = 1250
    maze = sess.paradigm["maze"].flatten()
    rpls = sess.ripple.time_slice(*maze)
    run = sess.maze_run.duration_slice(min_dur=0.2)
    # good_channels = sess.recinfo.good_channels.astype("int")
    # maze = sess.paradigm["maze"].flatten()
    # maze_lfp = sess.eegfile.get_signal(good_channels, *maze)
    # theta_stat = hilbert_ampltiude_stat(
    #     maze_lfp.traces, freq_band=(5, 16), fs=fs, statistic="mean"
    # )
    # best_theta_chan = maze_lfp.channel_id[theta_stat.argmax()]
    best_theta_chan = sess.ripple.metadata["channels"][3]
    # theta_lfp = sess.eegfile.get_signal(best_theta_chan,*maze)
    # theta_amp = np.abs(hilbertfast(filter_sig.bandpass(theta_lfp,lf=5,hf=12).traces[0]))
    # high_theta = stats.zscore(theta_amp)

    theta, theta_t = sess.eegfile.get_frames_within_epochs(
        run, best_theta_chan, ret_time=True
    )
    theta = theta.squeeze()
    theta_params = ThetaParams(theta, 1250)
    theta_phase = theta_params.angle

    spikes_in_theta = [
        _[np.digitize(_, bins=run.flatten()) % 2 == 1] for _ in neurons.spiketrains
    ]
    spikes_theta_phase = [np.interp(_, theta_t, theta_phase) for _ in spikes_in_theta]
    theta_bins = np.linspace(0, 360, 40)
    spikes_theta_hist = np.array(
        [np.histogram(_, bins=theta_bins)[0] for _ in spikes_theta_phase],
        dtype="object",
    )
    theta_neurons = Neurons(
        spikes_in_theta, t_start=maze[0], t_stop=maze[1], sampling_rate=30000
    )
    sort_indx = np.argsort(spikes_theta_hist.argmax(axis=1))
    theta_neurons = theta_neurons[sort_indx]
    # theta_frate = neurons.get_modulation_in_epochs(run, 1)

In [ ]:
_, axs = plt.subplots(2, 1, sharex=True, gridspec_kw=dict(height_ratios=[1, 5]))

ax = axs[0]
ax.plot(theta_t, theta)
ax.axis("off")

ax = axs[1]

plotting.plot_raster(theta_neurons, ax=ax, color="k")

In [ ]:
_, axs = plt.subplots(7, 7)
axs = axs.reshape(-1)
for i, h in enumerate(spikes_theta_hist):
    axs[i].plot(theta_bins[:-1], h)